In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
print(X.shape)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

(442, 10)


In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
# loss_fn = nn.L1Loss()
# loss_fn = nn.NLLLoss(reduction='mean')
opt = torch_numopt.GaussNewton(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.1, line_search_method='backtrack', line_search_cond='armijo')
# opt = torch_numopt.GaussNewton(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='wolfe')
# opt = torch_numopt.GaussNewton(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='strong-wolfe')
# opt = torch_numopt.GaussNewton(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='goldstein')

all_loss = {}
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch+1] += loss
    all_loss[epoch+1] /= len(data_loader)
    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

epoch:  0, loss: 0.4598602056503296
epoch:  1, loss: 0.459220290184021
epoch:  2, loss: 0.4265614449977875
epoch:  3, loss: 0.40943145751953125
epoch:  4, loss: 0.3929181694984436
epoch:  5, loss: 0.3746211528778076
epoch:  6, loss: 0.3568786680698395
epoch:  7, loss: 0.3395313322544098
epoch:  8, loss: 0.3224108815193176
epoch:  9, loss: 0.30596664547920227
epoch:  10, loss: 0.2890184223651886
epoch:  11, loss: 0.27232205867767334
epoch:  12, loss: 0.252121239900589
epoch:  13, loss: 0.23553621768951416
epoch:  14, loss: 0.22030159831047058
epoch:  15, loss: 0.20613881945610046
epoch:  16, loss: 0.19282163679599762
epoch:  17, loss: 0.1815483421087265
epoch:  18, loss: 0.16843508183956146
epoch:  19, loss: 0.15587976574897766
epoch:  20, loss: 0.14433977007865906
epoch:  21, loss: 0.13441333174705505
epoch:  22, loss: 0.13338324427604675
epoch:  23, loss: 0.12291105091571808
epoch:  24, loss: 0.11294158548116684
epoch:  25, loss: 0.10494914650917053
epoch:  26, loss: 0.096623905003070